In [1]:
#EXECUTE THIS COMMAND WITH KERNAL SET TO "conda_mxnet_p36". ONLY THIS KERNALCAN INSTALL EXTERNAL LIBRARIES.

! pip install --user ctgan

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/mxnet_p36/bin/python -m pip install --upgrade pip' command.


In [1]:
#import packages

from ctgan import CTGANSynthesizer
import pandas as pd
import boto3

In [2]:
#import source file

s3 = boto3.client('s3')

bucket = 'flood-prediction-master-dataset'
key = 'ml-sensor-data-15-min/ml_sensor_data_15_min.csv'

obj = s3.get_object(Bucket= bucket,Key= key)

file = pd.read_csv(obj['Body'],names=["time","river","rain","temperature","wind_direction","wind_speed"])

file

,time,river,rain,temperature,wind_direction,wind_speed
0,2020-07-11T14:45,7.659,0.0,18.37,270.0,6.433
1,2020-07-11T14:30,7.578,0.0,17.46,267.0,6.048
2,2020-07-11T14:15,7.442,0.0,16.35,268.0,6.334
3,2020-07-11T14:00,7.252,0.0,16.41,267.0,6.108
4,2020-07-11T13:45,7.019,0.0,16.83,268.0,5.325
...,...,...,...,...,...,...
2753,2020-06-12T01:00,6.078,0.0,11.94,4.0,4.914
2754,2020-06-12T00:45,5.702,0.0,11.84,5.0,5.201
2755,2020-06-12T00:30,5.332,0.0,11.71,8.0,5.099
2756,2020-06-12T00:15,4.932,0.0,11.61,4.0,5.554


In [4]:
#specify the headers to GAN

discrete_columns = [
    'time',
    'river',
    'rain',
    'temperature',
    'wind_direction',
    'wind_speed'
]

In [5]:
#creating instance

ctgan = CTGANSynthesizer()

In [6]:
#training

ctgan.fit(file, discrete_columns)

Epoch 1, Loss G: 6.5076, Loss D: -0.0168
Epoch 2, Loss G: 6.3924, Loss D: -0.0505
Epoch 3, Loss G: 6.5633, Loss D: -0.0877
Epoch 4, Loss G: 6.5609, Loss D: -0.1126
Epoch 5, Loss G: 6.5976, Loss D: -0.1468
Epoch 6, Loss G: 6.5404, Loss D: -0.1767
Epoch 7, Loss G: 6.4405, Loss D: -0.1780
Epoch 8, Loss G: 6.3071, Loss D: -0.1633
Epoch 9, Loss G: 6.3667, Loss D: -0.1390
Epoch 10, Loss G: 6.3521, Loss D: -0.1256
Epoch 11, Loss G: 6.3622, Loss D: -0.1235
Epoch 12, Loss G: 6.2453, Loss D: -0.0958
Epoch 13, Loss G: 6.2343, Loss D: -0.1127
Epoch 14, Loss G: 6.3644, Loss D: -0.0780
Epoch 15, Loss G: 6.3937, Loss D: -0.0471
Epoch 16, Loss G: 6.3570, Loss D: -0.0339
Epoch 17, Loss G: 6.4664, Loss D: -0.0477
Epoch 18, Loss G: 6.2243, Loss D: -0.0577
Epoch 19, Loss G: 6.4095, Loss D: -0.0655
Epoch 20, Loss G: 6.1731, Loss D: -0.0443
Epoch 21, Loss G: 6.4040, Loss D: -0.0612
Epoch 22, Loss G: 6.2055, Loss D: -0.0365
Epoch 23, Loss G: 6.1212, Loss D: -0.0222
Epoch 24, Loss G: 6.2856, Loss D: -0.0304
E

In [7]:
#sample creation

samples = ctgan.sample(2758)

In [8]:
import datetime
import tarfile

import boto3 # AWS SDK for python. Provides low-level access to AWS services
from sagemaker import get_execution_role
import sagemaker

m_boto3 = boto3.client('sagemaker') 

sess = sagemaker.Session()

region = sess.boto_session.region_name

bucket = 'flood-prediction-master-dataset'  #  specify the S3 bucket to save the file

print('Using bucket ' + bucket)

Using bucket flood-prediction-master-dataset


In [9]:
#save the file as csv locally

samples.to_csv("gan_data_15_min.csv")

In [10]:
# send data to S3. key_prefix is the directory path. path is the local file name which will be saved in S3 with same file name
# and bucket is the bucket name 

upload = sess.upload_data(path='gan_data_15_min.csv', bucket=bucket, key_prefix='ml-sensor-data-15-min/')